# Análisis de Documentos PDF: Extrayendo Historia de los Archivos

```{epigraph}
"La revolución cognitiva es el punto en que la historia se independizó de la biología. Hasta entonces, las acciones de todos los organismos estaban determinadas por sus genes. A partir de la revolución cognitiva, los relatos empezaron a determinar lo que hacían los sapiens."

-- Yuval Noah Harari, *Homo Deus* (2016)
```

```{admonition} Información del Capítulo
:class: tip

**Tiempo estimado:** 100 minutos  
**Nivel:** Intermedio-Avanzado  
**Prerrequisitos:** Capítulo 24 - Tokenización y Análisis de Texto
```

## Objetivos de Aprendizaje

Al finalizar este capítulo, serás capaz de:

1. Leer y extraer texto de documentos **PDF** usando Python
2. Aplicar técnicas de **tokenización y stopwords** a documentos completos
3. Crear funciones de **análisis automatizado** para discursos históricos
4. Buscar **palabras clave y frases** en documentos extensos
5. Generar **estadísticas y visualizaciones** del contenido textual

---

## Introducción: Los Archivos PDF en la Investigación Histórica

El formato **PDF** (*Portable Document Format*) se ha convertido en el estándar para la digitalización de documentos históricos. Bibliotecas, archivos nacionales y museos de todo el mundo han digitalizado millones de documentos en este formato.

Para los historiadores digitales, la capacidad de **extraer y analizar texto de PDFs automáticamente** representa una revolución metodológica:

- **Análisis de corpus extensos**: Miles de documentos procesados en horas
- **Búsqueda sistemática**: Localizar términos específicos en archivos completos
- **Comparación de discursos**: Analizar cambios en el lenguaje político a través del tiempo
- **Detección de patrones**: Identificar temas recurrentes en documentación histórica

```{admonition} Nuestro documento de estudio
:class: note

En este capítulo trabajaremos con el **Discurso Inaugural de Ronald Reagan** (20 de enero de 1981), un documento fundamental para entender el giro conservador en la política estadounidense de la década de 1980 y su impacto global durante la Guerra Fría.
```

### Contexto Histórico: El Discurso Inaugural de Reagan (1981)

**Ronald Reagan** (1911-2004) asumió la presidencia de Estados Unidos el 20 de enero de 1981, en un momento de crisis económica y tensión con la Unión Soviética. Su discurso inaugural marcó un cambio paradigmático en la política estadounidense:

- **Crítica al "gobierno grande"**: La famosa frase "el gobierno no es la solución a nuestro problema; el gobierno es el problema"
- **Renovación del patriotismo**: Llamado a recuperar la grandeza americana
- **Política económica**: Anuncio de reformas fiscales y reducción del Estado
- **Postura internacional**: Firmeza frente a la Unión Soviética

Este discurso es considerado uno de los más influyentes del siglo XX y estableció las bases del "reaganismo" como corriente política {cite}`reagan1981inaugural`.

---

## Preparación del Entorno

Para trabajar con PDFs en Python, utilizaremos la biblioteca **PyPDF2**, una herramienta potente y fácil de usar para extraer texto de documentos PDF {cite}`pypdf2docs`.

In [1]:
# Instalación de PyPDF2 (ejecutar solo si no está instalado)
# !pip install PyPDF2

In [2]:
# Importamos las bibliotecas necesarias
import PyPDF2
from PyPDF2 import PdfReader
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Descargamos recursos de NLTK si es necesario
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

print("✓ Bibliotecas importadas correctamente")

✓ Bibliotecas importadas correctamente


---

## Leyendo Documentos PDF

### Método básico: PdfReader

La clase `PdfReader` de PyPDF2 nos permite abrir un archivo PDF y extraer su contenido página por página.

In [3]:
def leer_pdf(ruta_archivo):
    """
    Lee un archivo PDF y extrae todo su texto.
    
    Parámetros:
    -----------
    ruta_archivo : str
        Ruta al archivo PDF
    
    Retorna:
    --------
    str : Texto completo del documento
    """
    try:
        # Abrimos el archivo PDF
        with open(ruta_archivo, 'rb') as archivo:
            # Creamos el lector de PDF
            lector = PdfReader(archivo)
            
            # Variable para almacenar todo el texto
            texto_completo = ""
            
            # Iteramos por cada página
            for num_pagina, pagina in enumerate(lector.pages):
                texto_pagina = pagina.extract_text()
                texto_completo += texto_pagina + "\n"
            
            return texto_completo, len(lector.pages)
            
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{ruta_archivo}'")
        return None, 0
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        return None, 0

### El Discurso de Reagan: Texto para Análisis

Para este ejercicio, trabajaremos con el texto del discurso inaugural de Reagan. El siguiente es un fragmento significativo del discurso original traducido al español:

In [4]:
# Discurso Inaugural de Ronald Reagan (20 de enero de 1981)
# Traducción al español de fragmentos clave

discurso_reagan = """
Los negocios de nuestra nación siguen adelante. Estados Unidos se enfrenta a una 
crisis económica de enormes proporciones. Sufrimos la inflación sostenida más larga 
y una de las peores de nuestra historia. Distorsiona nuestras decisiones económicas, 
penaliza el ahorro y aplasta por igual a los jóvenes con dificultades económicas y 
a los ancianos con ingresos fijos. Amenaza con destrozar la vida de millones de 
nuestros ciudadanos.

Las industrias ociosas han sumido a los trabajadores en el desempleo, la miseria y 
la indignidad personal. A quienes trabajan se les niega una remuneración justa por 
su labor debido a un sistema tributario que penaliza el éxito y nos impide mantener 
la plena productividad.

En la crisis actual, el gobierno no es la solución a nuestro problema; el gobierno 
es el problema. De vez en cuando hemos caído en la tentación de creer que la sociedad 
se ha vuelto demasiado compleja para ser gestionada por el autogobierno, que el 
gobierno de una élite es superior al gobierno para, por y del pueblo.

Se habla mucho de grupos de interés. Pues bien, nuestra preocupación debe ser por 
un grupo de interés que ha sido desatendido durante demasiado tiempo. No conoce 
fronteras sectoriales ni divisiones étnicas ni raciales, y trasciende las fronteras 
de los partidos políticos. Está formado por hombres y mujeres que cultivan nuestros 
alimentos, patrullan nuestras calles, trabajan en nuestras minas y fábricas, educan 
a nuestros hijos, mantienen nuestros hogares y nos curan cuando enfermamos: 
profesionales, industriales, comerciantes, oficinistas, taxistas y camioneros. 
En resumen, son «Nosotros, el pueblo», esta raza llamada estadounidenses.

El objetivo de esta administración será una economía sana, vigorosa y en crecimiento 
que brinde igualdad de oportunidades a todos los estadounidenses, sin barreras 
derivadas de la intolerancia ni la discriminación. Poner a Estados Unidos a trabajar 
de nuevo significa que todos los estadounidenses vuelvan a trabajar. Acabar con la 
inflación significa liberar a todos los estadounidenses del terror del descontrol 
del costo de vida.

Somos una nación con gobierno, no al revés. Y esto nos hace especiales entre las 
naciones de la Tierra. Nuestro gobierno no tiene más poder que el que le otorga el 
pueblo. Es hora de frenar y revertir el crecimiento del gobierno, que muestra signos 
de haber crecido sin el consentimiento de los gobernados.

Mi intención es limitar el tamaño y la influencia del establishment federal y exigir 
que se reconozca la distinción entre los poderes otorgados al Gobierno Federal y los 
reservados a los Estados o al pueblo. Es necesario recordar a todos que el Gobierno 
Federal no creó a los Estados; los Estados crearon al Gobierno Federal.

Si buscamos la respuesta a por qué durante tantos años logramos tanto y prosperamos 
como ningún otro pueblo de la Tierra, fue porque aquí, en esta tierra, desatamos la 
energía y el genio individual del hombre como nunca antes. La libertad y la dignidad 
del individuo han estado más disponibles y garantizadas aquí que en cualquier otro 
lugar de la Tierra.

Es hora de que nos demos cuenta de que somos una nación demasiado grande como para 
limitarnos a pequeños sueños. No estamos, como algunos quieren hacernos creer, 
condenados a una decadencia inevitable. No creo en un destino que nos caerá encima 
hagamos lo que hagamos. Sí creo en un destino que nos caerá encima si no hacemos nada.

Tenemos todo el derecho a soñar con héroes. Quienes dicen que vivimos en una época 
donde no hay héroes, simplemente no saben dónde buscar. Se ven héroes a diario 
entrando y saliendo de las fábricas. Hay emprendedores con fe en sí mismos y en una 
idea que crean nuevos empleos, nueva riqueza y oportunidades.

Con aquellos vecinos y aliados que comparten nuestra libertad, fortaleceremos nuestros 
lazos históricos y les aseguraremos nuestro apoyo y firme compromiso. En cuanto a los 
enemigos de la libertad, aquellos que son adversarios potenciales, se les recordará 
que la paz es la mayor aspiración del pueblo estadounidense. Negociaremos por ella, 
nos sacrificaremos por ella; no nos rendiremos por ella, ni ahora ni nunca.

Nuestra paciencia nunca debe malinterpretarse. Nuestra reticencia al conflicto no 
debe interpretarse erróneamente como una falta de voluntad. Cuando sea necesario 
actuar para preservar nuestra seguridad nacional, actuaremos. Mantendremos la fuerza 
suficiente para prevalecer si es necesario, sabiendo que, si lo hacemos, tendremos 
la mejor oportunidad de no tener que usarla jamás.

Sobre todo, debemos comprender que ningún arsenal ni arma en los arsenales del mundo 
es tan formidable como la voluntad y el coraje moral de hombres y mujeres libres.
"""

print(f"Longitud del discurso: {len(discurso_reagan):,} caracteres")

Longitud del discurso: 4,759 caracteres


---

## Análisis Completo del Discurso

Ahora aplicaremos todas las técnicas aprendidas en el capítulo anterior para analizar el discurso de Reagan.

In [5]:
def analizar_discurso_historico(texto, titulo, idioma='spanish', n_palabras=20):
    """
    Realiza un análisis completo de un discurso histórico.
    
    Parámetros:
    -----------
    texto : str
        El texto del discurso
    titulo : str
        Título del documento
    idioma : str
        Idioma para stopwords ('spanish' o 'english')
    n_palabras : int
        Número de palabras frecuentes a mostrar
    
    Retorna:
    --------
    dict : Diccionario con todos los resultados del análisis
    """
    # Configuración de stopwords
    stop_words = set(stopwords.words(idioma))
    
    # Stopwords adicionales para discursos políticos
    stopwords_politicas = {
        'señores', 'señoras', 'ustedes', 'nosotros', 'nuestro', 'nuestra',
        'nuestros', 'nuestras', 'hoy', 'ahora', 'bien', 'así', 'hace',
        'hacer', 'sido', 'ser', 'puede', 'pueden', 'debe', 'deben',
        'toda', 'todo', 'todos', 'todas', 'cada', 'más', 'menos'
    }
    stop_words = stop_words.union(stopwords_politicas)
    
    # Tokenización
    oraciones = sent_tokenize(texto, language=idioma)
    palabras = word_tokenize(texto, language=idioma)
    solo_palabras = [p for p in palabras if p.isalpha()]
    palabras_lower = [p.lower() for p in solo_palabras]
    
    # Filtrado de stopwords
    palabras_filtradas = [p for p in palabras_lower if p not in stop_words]
    
    # Frecuencias
    frecuencias = Counter(palabras_filtradas)
    palabras_unicas = set(palabras_lower)
    
    # Cálculos adicionales
    promedio_palabras = len(solo_palabras) / len(oraciones) if oraciones else 0
    riqueza_lexica = len(palabras_unicas) / len(palabras_lower) if palabras_lower else 0
    
    # Mostrar resultados
    print("\n" + "═"*70)
    print(f" ANÁLISIS DEL DISCURSO: {titulo.upper()}")
    print("═"*70)
    
    print("\n📊 ESTADÍSTICAS GENERALES")
    print("-"*40)
    print(f"   Caracteres totales:     {len(texto):,}")
    print(f"   Número de oraciones:    {len(oraciones)}")
    print(f"   Palabras totales:       {len(solo_palabras):,}")
    print(f"   Palabras únicas:        {len(palabras_unicas):,}")
    print(f"   Promedio palabras/oración: {promedio_palabras:.1f}")
    print(f"   Riqueza léxica:         {riqueza_lexica:.2%}")
    
    print(f"\n📝 TOP {n_palabras} PALABRAS MÁS FRECUENTES")
    print("-"*40)
    for i, (palabra, freq) in enumerate(frecuencias.most_common(n_palabras), 1):
        barra = "█" * min(freq, 15)
        print(f"   {i:2}. {palabra.ljust(15)} {barra} ({freq})")
    
    # Retornamos los datos
    return {
        'titulo': titulo,
        'caracteres': len(texto),
        'oraciones': oraciones,
        'num_oraciones': len(oraciones),
        'palabras_totales': len(solo_palabras),
        'palabras_unicas': len(palabras_unicas),
        'promedio_palabras': promedio_palabras,
        'riqueza_lexica': riqueza_lexica,
        'frecuencias': frecuencias,
        'palabras_filtradas': palabras_filtradas
    }

In [6]:
# Analizamos el discurso de Reagan
resultado_reagan = analizar_discurso_historico(
    texto=discurso_reagan,
    titulo="Discurso Inaugural de Ronald Reagan (1981)",
    idioma='spanish',
    n_palabras=15
)


══════════════════════════════════════════════════════════════════════
 ANÁLISIS DEL DISCURSO: DISCURSO INAUGURAL DE RONALD REAGAN (1981)
══════════════════════════════════════════════════════════════════════

📊 ESTADÍSTICAS GENERALES
----------------------------------------
   Caracteres totales:     4,759
   Número de oraciones:    41
   Palabras totales:       750
   Palabras únicas:        363
   Promedio palabras/oración: 18.3
   Riqueza léxica:         48.40%

📝 TOP 15 PALABRAS MÁS FRECUENTES
----------------------------------------
    1. gobierno        ██████████ (10)
    2. pueblo          ██████ (6)
    3. estadounidenses ████ (4)
    4. tierra          ████ (4)
    5. federal         ████ (4)
    6. si              ████ (4)
    7. nación          ███ (3)
    8. demasiado       ███ (3)
    9. necesario       ███ (3)
   10. nunca           ███ (3)
   11. libertad        ███ (3)
   12. héroes          ███ (3)
   13. unidos          ██ (2)
   14. crisis          ██ (2)
   15. 

```{admonition} Interpretación Histórica
:class: seealso

Las palabras más frecuentes en el discurso de Reagan revelan sus **ejes temáticos principales**:

- **"gobierno"**: Tema central del discurso, criticado como "el problema"
- **"libertad"**: Valor fundamental del conservadurismo reaganiano
- **"pueblo"**: Apelación populista a "Nosotros, el pueblo"
- **"nación"**: Discurso nacionalista y patriótico
- **"Estados Unidos"**: Excepcionalismo americano

Este análisis cuantitativo confirma la interpretación histórica: Reagan construyó un discurso que contraponía al "pueblo trabajador" contra el "gobierno grande", estableciendo las bases ideológicas del neoliberalismo estadounidense.
```

---

## Búsqueda de Palabras Clave y Frases

Una tarea esencial en el análisis de documentos históricos es buscar términos específicos y analizar su contexto.

In [7]:
def buscar_palabras_clave(texto, palabras_clave, contexto=80):
    """
    Busca múltiples palabras clave en un texto y muestra estadísticas.
    
    Parámetros:
    -----------
    texto : str
        El texto donde buscar
    palabras_clave : list
        Lista de palabras a buscar
    contexto : int
        Caracteres de contexto a mostrar
    """
    texto_lower = texto.lower()
    
    print("\n" + "="*60)
    print(" BÚSQUEDA DE PALABRAS CLAVE")
    print("="*60)
    
    resultados = {}
    
    for palabra in palabras_clave:
        palabra_lower = palabra.lower()
        ocurrencias = texto_lower.count(palabra_lower)
        resultados[palabra] = ocurrencias
        
        print(f"\n🔍 '{palabra}': {ocurrencias} ocurrencias")
        
        if ocurrencias > 0:
            # Mostramos la primera ocurrencia con contexto
            pos = texto_lower.find(palabra_lower)
            inicio = max(0, pos - contexto)
            fin = min(len(texto), pos + len(palabra) + contexto)
            fragmento = texto[inicio:fin].replace('\n', ' ').strip()
            print(f"   Contexto: ...{fragmento}...")
    
    return resultados

In [8]:
# Palabras clave relevantes para el análisis del discurso de Reagan
palabras_clave_reagan = [
    'gobierno',
    'libertad',
    'pueblo',
    'crisis',
    'inflación',
    'economía',
    'héroes',
    'Estados',
    'paz',
    'enemigos'
]

resultados_busqueda = buscar_palabras_clave(discurso_reagan, palabras_clave_reagan)


 BÚSQUEDA DE PALABRAS CLAVE

🔍 'gobierno': 11 ocurrencias
   Contexto: ...l éxito y nos impide mantener  la plena productividad.  En la crisis actual, el gobierno no es la solución a nuestro problema; el gobierno  es el problema. De vez en cu...

🔍 'libertad': 3 ocurrencias
   Contexto: ...ra, desatamos la  energía y el genio individual del hombre como nunca antes. La libertad y la dignidad  del individuo han estado más disponibles y garantizadas aquí que...

🔍 'pueblo': 6 ocurrencias
   Contexto: ...gobierno, que el  gobierno de una élite es superior al gobierno para, por y del pueblo.  Se habla mucho de grupos de interés. Pues bien, nuestra preocupación debe ser...

🔍 'crisis': 2 ocurrencias
   Contexto: ...s negocios de nuestra nación siguen adelante. Estados Unidos se enfrenta a una  crisis económica de enormes proporciones. Sufrimos la inflación sostenida más larga  y...

🔍 'inflación': 2 ocurrencias
   Contexto: ...Unidos se enfrenta a una  crisis económica de enormes proporcion

---

## Análisis de Oraciones Específicas

Podemos buscar las oraciones que contienen ciertos términos para un análisis más profundo.

In [9]:
def encontrar_oraciones_con_palabra(texto, palabra, idioma='spanish', max_oraciones=5):
    """
    Encuentra todas las oraciones que contienen una palabra específica.
    
    Parámetros:
    -----------
    texto : str
        El texto a analizar
    palabra : str
        La palabra a buscar
    idioma : str
        Idioma para tokenización
    max_oraciones : int
        Máximo de oraciones a mostrar
    """
    oraciones = sent_tokenize(texto, language=idioma)
    palabra_lower = palabra.lower()
    
    oraciones_encontradas = []
    for oracion in oraciones:
        if palabra_lower in oracion.lower():
            oraciones_encontradas.append(oracion.strip())
    
    print(f"\n📖 Oraciones que contienen '{palabra}':")
    print(f"   (Encontradas: {len(oraciones_encontradas)} de {len(oraciones)} oraciones)")
    print("-"*60)
    
    for i, oracion in enumerate(oraciones_encontradas[:max_oraciones], 1):
        # Limpiamos la oración
        oracion_limpia = ' '.join(oracion.split())
        print(f"\n   {i}. {oracion_limpia}")
    
    if len(oraciones_encontradas) > max_oraciones:
        print(f"\n   ... y {len(oraciones_encontradas) - max_oraciones} oraciones más.")
    
    return oraciones_encontradas

In [10]:
# Buscamos oraciones con palabras clave del discurso de Reagan
oraciones_gobierno = encontrar_oraciones_con_palabra(discurso_reagan, "gobierno")
oraciones_libertad = encontrar_oraciones_con_palabra(discurso_reagan, "libertad")


📖 Oraciones que contienen 'gobierno':
   (Encontradas: 7 de 41 oraciones)
------------------------------------------------------------

   1. En la crisis actual, el gobierno no es la solución a nuestro problema; el gobierno es el problema.

   2. De vez en cuando hemos caído en la tentación de creer que la sociedad se ha vuelto demasiado compleja para ser gestionada por el autogobierno, que el gobierno de una élite es superior al gobierno para, por y del pueblo.

   3. Somos una nación con gobierno, no al revés.

   4. Nuestro gobierno no tiene más poder que el que le otorga el pueblo.

   5. Es hora de frenar y revertir el crecimiento del gobierno, que muestra signos de haber crecido sin el consentimiento de los gobernados.

   ... y 2 oraciones más.

📖 Oraciones que contienen 'libertad':
   (Encontradas: 3 de 41 oraciones)
------------------------------------------------------------

   1. La libertad y la dignidad del individuo han estado más disponibles y garantizadas aquí que en

---

## Comparación de Discursos: Reagan vs. Bello

Una aplicación poderosa del análisis de texto es la **comparación entre documentos históricos** de diferentes épocas y contextos.

In [11]:
# Fragmento del discurso de Andrés Bello para comparación
discurso_bello_fragmento = """
La Universidad, señores, no sería digna de ocupar un lugar en nuestras instituciones 
sociales, si el cultivo de las ciencias y de las letras pudiese mirarse como peligroso 
bajo un punto de vista moral, o bajo un punto de vista político. La moral es la vida 
misma de la sociedad; la libertad es el estímulo que da un vigor sano y una actividad 
fecunda a las instituciones sociales.

Las ciencias y las letras, fuera de ese valor social, fuera de esta importancia que 
podemos llamar instrumental, tienen un mérito suyo, intrínseco, en cuanto aumentan 
los placeres y goces del individuo que las cultiva y las ama.

La Universidad estudiará también las especialidades de la sociedad chilena bajo el 
punto de vista económico. Porque en este, como en los otros ramos, el programa de la 
Universidad es enteramente chileno: si toma prestadas a la Europa las deducciones 
de la ciencia, es para aplicarlas a Chile. Todas las sendas convergen a un centro: 
la patria.
"""

# Analizamos ambos discursos
print("\n" + "="*70)
print(" COMPARACIÓN DE DISCURSOS HISTÓRICOS")
print("="*70)

resultado_bello = analizar_discurso_historico(
    texto=discurso_bello_fragmento,
    titulo="Discurso de Andrés Bello (1843)",
    n_palabras=10
)


 COMPARACIÓN DE DISCURSOS HISTÓRICOS

══════════════════════════════════════════════════════════════════════
 ANÁLISIS DEL DISCURSO: DISCURSO DE ANDRÉS BELLO (1843)
══════════════════════════════════════════════════════════════════════

📊 ESTADÍSTICAS GENERALES
----------------------------------------
   Caracteres totales:     967
   Número de oraciones:    6
   Palabras totales:       163
   Palabras únicas:        92
   Promedio palabras/oración: 27.2
   Riqueza léxica:         56.44%

📝 TOP 10 PALABRAS MÁS FRECUENTES
----------------------------------------
    1. universidad     ███ (3)
    2. bajo            ███ (3)
    3. punto           ███ (3)
    4. vista           ███ (3)
    5. instituciones   ██ (2)
    6. sociales        ██ (2)
    7. si              ██ (2)
    8. ciencias        ██ (2)
    9. letras          ██ (2)
   10. moral           ██ (2)


In [12]:
# Tabla comparativa
print("\n" + "="*70)
print(" TABLA COMPARATIVA: BELLO vs REAGAN")
print("="*70)
print(f"\n{'Métrica':<30} {'Bello (1843)':<20} {'Reagan (1981)':<20}")
print("-"*70)
print(f"{'Año del discurso':<30} {'1843':<20} {'1981':<20}")
print(f"{'País':<30} {'Chile':<20} {'Estados Unidos':<20}")
print(f"{'Ocasión':<30} {'Universidad':<20} {'Inauguración':<20}")
print(f"{'Oraciones':<30} {resultado_bello['num_oraciones']:<20} {resultado_reagan['num_oraciones']:<20}")
print(f"{'Palabras totales':<30} {resultado_bello['palabras_totales']:<20,} {resultado_reagan['palabras_totales']:<20,}")
print(f"{'Palabras únicas':<30} {resultado_bello['palabras_unicas']:<20,} {resultado_reagan['palabras_unicas']:<20,}")
print(f"{'Promedio palabras/oración':<30} {resultado_bello['promedio_palabras']:<20.1f} {resultado_reagan['promedio_palabras']:<20.1f}")
print(f"{'Riqueza léxica':<30} {resultado_bello['riqueza_lexica']:<20.2%} {resultado_reagan['riqueza_lexica']:<20.2%}")


 TABLA COMPARATIVA: BELLO vs REAGAN

Métrica                        Bello (1843)         Reagan (1981)       
----------------------------------------------------------------------
Año del discurso               1843                 1981                
País                           Chile                Estados Unidos      
Ocasión                        Universidad          Inauguración        
Oraciones                      6                    41                  
Palabras totales               163                  750                 
Palabras únicas                92                   363                 
Promedio palabras/oración      27.2                 18.3                
Riqueza léxica                 56.44%               48.40%              


```{admonition} Interpretación Comparativa
:class: important

La comparación entre los discursos de **Bello (1843)** y **Reagan (1981)** revela diferencias estilísticas y temáticas significativas:

**Diferencias estilísticas:**
- Bello usa oraciones más largas y elaboradas (estilo académico del siglo XIX)
- Reagan emplea oraciones más cortas y directas (retórica política moderna)

**Diferencias temáticas:**
- **Bello**: Ciencias, letras, universidad, moral, patria → Enfoque en educación y conocimiento
- **Reagan**: Gobierno, libertad, pueblo, crisis, economía → Enfoque en política y economía

**Similitudes:**
- Ambos apelan al concepto de "libertad"
- Ambos tienen un fuerte componente nacionalista ("patria", "nación")
- Ambos buscan definir el rol del Estado en la sociedad
```

---

## Actividad de Cierre

```{admonition} Instrucciones
:class: important

Realiza los siguientes ejercicios para practicar el análisis de documentos PDF y textos históricos.
```

### Ejercicio 1: Análisis de frases célebres

La frase más famosa del discurso de Reagan es: *"El gobierno no es la solución a nuestro problema; el gobierno es el problema."*

Busca esta frase en el texto y analiza las 3 oraciones que la rodean (anterior y posterior) para entender su contexto.

In [13]:
# Escribe tu código aquí


### Ejercicio 2: Palabras únicas de cada discurso

Encuentra las palabras que aparecen **solo en el discurso de Reagan** y **solo en el discurso de Bello** (excluyendo stopwords). ¿Qué revelan sobre cada orador?

In [14]:
# Escribe tu código aquí


### Ejercicio 3: Crea tu propia función de análisis

Crea una función llamada `generar_resumen_discurso()` que reciba un texto y devuelva:
- Las 3 oraciones más largas del discurso
- Las 3 oraciones más cortas del discurso
- La oración que contiene más palabras clave de una lista dada

In [15]:
# Escribe tu código aquí


---

## Resumen del Capítulo

| Concepto | Descripción | Biblioteca/Función |
|----------|-------------|--------------------|
| Lectura de PDF | Extraer texto de archivos PDF | `PyPDF2.PdfReader()` |
| Análisis de discursos | Estadísticas y frecuencias | Funciones personalizadas |
| Búsqueda de palabras | Localizar términos en textos | `str.count()`, `str.find()` |
| Comparación de textos | Analizar diferencias entre documentos | Counter, sets |

```{admonition} Proyecto Final Sugerido
:class: seealso

Como proyecto integrador, podrías:
1. Descargar 3-5 discursos presidenciales chilenos de diferentes épocas
2. Aplicar todas las técnicas aprendidas para analizarlos
3. Crear un informe comparativo que muestre la evolución del lenguaje político chileno
4. Identificar qué temas han sido constantes y cuáles han cambiado
```

---

## Referencias

- Bird, S., Klein, E., & Loper, E. (2009). *Natural Language Processing with Python*. O'Reilly Media.
- Harari, Y. N. (2016). *Homo Deus: Breve historia del mañana*. Debate.
- PyPDF2 Documentation. (2024). *PyPDF2 - A Python library to work with PDF files*. https://pypdf2.readthedocs.io/
- Reagan, R. (1981). *Inaugural Address*. The American Presidency Project. https://www.presidency.ucsb.edu/documents/inaugural-address-11
- Reagan Presidential Library. (2024). *Speeches and Addresses*. https://www.reaganlibrary.gov/archives/speech/inaugural-address-1981